In [1]:
# Import findspark and initialise. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

24/05/10 05:31:10 WARN Utils: Your hostname, Bens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.179 instead (on interface en0)
24/05/10 05:31:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/10 05:31:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')

home_sales_df.show()


24/05/10 05:31:41 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [9]:
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [15]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS Average_price 
    FROM home_sales
    GROUP BY Year
    ORDER BY Year
"""
).show()


+----+-------------+
|Year|Average_price|
+----+-------------+
|2019|    315782.18|
|2020|    315949.81|
|2021|    313218.09|
|2022|    314421.54|
+----+-------------+



In [16]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT date_built AS Year_built, ROUND(AVG(price), 2) AS Average_price 
    FROM home_sales
    GROUP BY Year_built
    ORDER BY Year_built
"""
).show()


+----------+-------------+
|Year_built|Average_price|
+----------+-------------+
|      2010|    309751.17|
|      2011|    314804.32|
|      2012|    314072.77|
|      2013|     317597.6|
|      2014|    316395.73|
|      2015|    315322.64|
|      2016|    315471.57|
|      2017|    315799.06|
+----------+-------------+



In [21]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT date_built AS Year_built, ROUND(AVG(price), 2) AS Average_price 
    FROM home_sales
    WHERE bedrooms = 3 AND
        bathrooms = 3 AND
        floors = 2 AND
        sqft_lot >= 2000
    GROUP BY Year_built
    ORDER BY Year_built
"""
).show()


+----------+-------------+
|Year_built|Average_price|
+----------+-------------+
|      2010|    280447.23|
|      2011|    281413.45|
|      2012|    295858.68|
|      2013|    295142.13|
|      2014|    294195.13|
|      2015|    291788.36|
|      2016|     287812.6|
|      2017|    282026.59|
+----------+-------------+



In [22]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS Average_price 
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
"""
).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.19574189186096191 seconds ---


In [24]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [25]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [26]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS Average_price 
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
"""
).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|Average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.13165688514709473 seconds ---


In [27]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.parquet('parquet_home_sales', mode='overwrite')

In [28]:
# 11. Read the formatted parquet data.
p_home_sales_df = spark.read.parquet('parquet_home_sales')

In [29]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView('p_home_sales')

In [30]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS Average_price 
    FROM p_home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
"""
).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.23034000396728516 seconds ---


In [31]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [33]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False